In [ ]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import datetime
import dask.array
from shapely.geometry import Point
import snowFun

In [1]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_climate = os.path.join(folder_AGVA, "Climate")
folder_csv = os.path.join(folder_climate, "csv", 'daily')

In [2]:
# open rgi attribute table, subsetting to the columns we want
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_df = gpd.read_file(path_rgi, include_fields=["RGIId","CenLon","CenLat","Area","Zmed"])

# path_rgi = os.path.join(folder_AGVA, 'RGI', "01_rgi60_Alaska", "01_rgi60_Alaska.shp")
# # path_rgi = os.path.join(folder_AGVA, 'RGI', "RGI_v70", "RGI2000-v7.0-G-01_alaska", "RGI2000-v7.0-G-01_alaska-attributes.csv")
# rgi_df = pd.read_csv(path_rgi, usecols=["rgi_id","cenlon","cenlat","area_km2","zmed_m","zmean_m"])
print(rgi_df.shape)
rgi_df.head()

(3031, 6)


,RGIId,CenLon,CenLat,Area,Zmed,geometry
0,RGI60-01.00004,-146.120,63.381,3.681,1944,"POLYGON ((390852.892 1513270.533, 390859.011 1..."
1,RGI60-01.00005,-147.057,63.551,2.573,1914,"POLYGON ((344516.095 1527311.059, 344445.809 1..."
2,RGI60-01.00006,-146.244,63.571,10.470,1740,"POLYGON ((383858.051 1532357.663, 383852.553 1..."
3,RGI60-01.00010,-147.170,63.513,3.806,1666,"POLYGON ((339736.019 1522275.072, 339760.103 1..."
4,RGI60-01.00012,-146.651,63.366,2.802,1725,"POLYGON ((367276.652 1509860.919, 367276.151 1..."


In [3]:
# list the files in the csv folder
all_files = os.listdir(folder_csv)
print(len(all_files))
print(all_files[0])

3031
RGI60-01.00004.csv


In [4]:
### open each RGI's daily climate csv, calculate monthly average temperature, average daily precip, PDDs

# define year and month range
all_years = ["18","19","20","21","22"]
all_months = ["01","02","03","04","05","06","07","08","09","10","11","12"]

# list to hold all data
all_data = []

# iterate through files, opening each
c=0
for f in all_files:
    c+=1
    print(f"{c} of {len(all_files)}")
    
    # open df
    test_df = pd.read_csv(os.path.join(folder_csv, f))

    # add year, month column
    test_df['y'] = [i[2:4] for i in test_df['date']]
    test_df['m'] = [i[5:7] for i in test_df['date']]
    
    # add column for lapse-rate corrected temp, PDD
    test_df['t_lapse'] = test_df['t'] + (test_df['zmed_m'] - test_df['z_era5land'])*(1/1000)*-5.0
    test_df['pdd'] = (test_df['t_lapse']-273.15).clip(lower=0)

    rgi_data = {'RGIId' : f[:14],
                'z_era5land' : int(test_df['z_era5land'].values[0])}

    # go through each year/month combo, calculating average daily temp, precip, total PDD
    for y_i in all_years:

        # subset to these years
        subset_y = test_df[test_df['y']==y_i]

        for m_i in all_months:

            # subset again to months
            subset_ym = subset_y[subset_y['m']==m_i]

            # get average temp, precip, sum PDD
            t_avg = np.nanmean(subset_ym['t']).round(6)
            p_avg = np.nanmean(subset_ym['p']).round(6)
            pdd_sum = np.nansum(subset_ym['pdd']).round(6)

            # append to rgi_data
            rgi_data[f'{y_i}_{m_i}_p'] = p_avg
            rgi_data[f'{y_i}_{m_i}_t'] = t_avg
            rgi_data[f'{y_i}_{m_i}_pdd'] = pdd_sum
    
    # append to final data list
    all_data.append(rgi_data)

1 of 3031
2 of 3031
3 of 3031
4 of 3031
5 of 3031
6 of 3031
7 of 3031
8 of 3031
9 of 3031
10 of 3031
11 of 3031
12 of 3031
13 of 3031
14 of 3031
15 of 3031
16 of 3031
17 of 3031
18 of 3031
19 of 3031
20 of 3031
21 of 3031
22 of 3031
23 of 3031
24 of 3031
25 of 3031
26 of 3031
27 of 3031
28 of 3031
29 of 3031
30 of 3031
31 of 3031
32 of 3031
33 of 3031
34 of 3031
35 of 3031
36 of 3031
37 of 3031
38 of 3031
39 of 3031
40 of 3031
41 of 3031
42 of 3031
43 of 3031
44 of 3031
45 of 3031
46 of 3031
47 of 3031
48 of 3031
49 of 3031
50 of 3031
51 of 3031
52 of 3031
53 of 3031
54 of 3031
55 of 3031
56 of 3031
57 of 3031
58 of 3031
59 of 3031
60 of 3031
61 of 3031
62 of 3031
63 of 3031
64 of 3031
65 of 3031
66 of 3031
67 of 3031
68 of 3031
69 of 3031
70 of 3031
71 of 3031
72 of 3031
73 of 3031
74 of 3031
75 of 3031
76 of 3031
77 of 3031
78 of 3031
79 of 3031
80 of 3031
81 of 3031
82 of 3031
83 of 3031
84 of 3031
85 of 3031
86 of 3031
87 of 3031
88 of 3031
89 of 3031
90 of 3031
91 of 3031
92 of 30

694 of 3031
695 of 3031
696 of 3031
697 of 3031
698 of 3031
699 of 3031
700 of 3031
701 of 3031
702 of 3031
703 of 3031
704 of 3031
705 of 3031
706 of 3031
707 of 3031
708 of 3031
709 of 3031
710 of 3031
711 of 3031
712 of 3031
713 of 3031
714 of 3031
715 of 3031
716 of 3031
717 of 3031
718 of 3031
719 of 3031
720 of 3031
721 of 3031
722 of 3031
723 of 3031
724 of 3031
725 of 3031
726 of 3031
727 of 3031
728 of 3031
729 of 3031
730 of 3031
731 of 3031
732 of 3031
733 of 3031
734 of 3031
735 of 3031
736 of 3031
737 of 3031
738 of 3031
739 of 3031
740 of 3031
741 of 3031
742 of 3031
743 of 3031
744 of 3031
745 of 3031
746 of 3031
747 of 3031
748 of 3031
749 of 3031
750 of 3031
751 of 3031
752 of 3031
753 of 3031
754 of 3031
755 of 3031
756 of 3031
757 of 3031
758 of 3031
759 of 3031
760 of 3031
761 of 3031
762 of 3031
763 of 3031
764 of 3031
765 of 3031
766 of 3031
767 of 3031
768 of 3031
769 of 3031
770 of 3031
771 of 3031
772 of 3031
773 of 3031
774 of 3031
775 of 3031
776 of 3031
777 

C:\Users\lzell\AppData\Local\Temp\ipykernel_12480\812262311.py:42: RuntimeWarning: Mean of empty slice
  t_avg = np.nanmean(subset_ym['t']).round(6)
C:\Users\lzell\AppData\Local\Temp\ipykernel_12480\812262311.py:43: RuntimeWarning: Mean of empty slice
  p_avg = np.nanmean(subset_ym['p']).round(6)
C:\Users\lzell\AppData\Local\Temp\ipykernel_12480\812262311.py:42: RuntimeWarning: Mean of empty slice
  t_avg = np.nanmean(subset_ym['t']).round(6)
C:\Users\lzell\AppData\Local\Temp\ipykernel_12480\812262311.py:43: RuntimeWarning: Mean of empty slice
  p_avg = np.nanmean(subset_ym['p']).round(6)


949 of 3031
950 of 3031
951 of 3031
952 of 3031
953 of 3031
954 of 3031
955 of 3031
956 of 3031
957 of 3031
958 of 3031
959 of 3031
960 of 3031
961 of 3031
962 of 3031
963 of 3031
964 of 3031
965 of 3031
966 of 3031
967 of 3031
968 of 3031
969 of 3031
970 of 3031
971 of 3031
972 of 3031
973 of 3031
974 of 3031
975 of 3031
976 of 3031
977 of 3031
978 of 3031
979 of 3031
980 of 3031
981 of 3031
982 of 3031
983 of 3031
984 of 3031
985 of 3031
986 of 3031
987 of 3031
988 of 3031
989 of 3031
990 of 3031
991 of 3031
992 of 3031
993 of 3031
994 of 3031
995 of 3031
996 of 3031
997 of 3031
998 of 3031
999 of 3031
1000 of 3031
1001 of 3031
1002 of 3031
1003 of 3031
1004 of 3031
1005 of 3031
1006 of 3031
1007 of 3031
1008 of 3031
1009 of 3031
1010 of 3031
1011 of 3031
1012 of 3031
1013 of 3031
1014 of 3031
1015 of 3031
1016 of 3031
1017 of 3031
1018 of 3031
1019 of 3031
1020 of 3031
1021 of 3031
1022 of 3031
1023 of 3031
1024 of 3031
1025 of 3031
1026 of 3031
1027 of 3031
1028 of 3031
1029 of 303

C:\Users\lzell\AppData\Local\Temp\ipykernel_12480\812262311.py:42: RuntimeWarning: Mean of empty slice
  t_avg = np.nanmean(subset_ym['t']).round(6)
C:\Users\lzell\AppData\Local\Temp\ipykernel_12480\812262311.py:43: RuntimeWarning: Mean of empty slice
  p_avg = np.nanmean(subset_ym['p']).round(6)


...

C:\Users\lzell\AppData\Local\Temp\ipykernel_12480\812262311.py:42: RuntimeWarning: Mean of empty slice
  t_avg = np.nanmean(subset_ym['t']).round(6)
C:\Users\lzell\AppData\Local\Temp\ipykernel_12480\812262311.py:43: RuntimeWarning: Mean of empty slice
  p_avg = np.nanmean(subset_ym['p']).round(6)
C:\Users\lzell\AppData\Local\Temp\ipykernel_12480\812262311.py:42: RuntimeWarning: Mean of empty slice
  t_avg = np.nanmean(subset_ym['t']).round(6)
C:\Users\lzell\AppData\Local\Temp\ipykernel_12480\812262311.py:43: RuntimeWarning: Mean of empty slice
  p_avg = np.nanmean(subset_ym['p']).round(6)


2114 of 3031
2115 of 3031
2116 of 3031
2117 of 3031
2118 of 3031
2119 of 3031
2120 of 3031
2121 of 3031
2122 of 3031
2123 of 3031
2124 of 3031
2125 of 3031
2126 of 3031
2127 of 3031
2128 of 3031
2129 of 3031
2130 of 3031
2131 of 3031
2132 of 3031
2133 of 3031
2134 of 3031
2135 of 3031
2136 of 3031
2137 of 3031
2138 of 3031
2139 of 3031
2140 of 3031
2141 of 3031
2142 of 3031
2143 of 3031
2144 of 3031
2145 of 3031
2146 of 3031
2147 of 3031
2148 of 3031
2149 of 3031
2150 of 3031
2151 of 3031
2152 of 3031
2153 of 3031
2154 of 3031
2155 of 3031
2156 of 3031
2157 of 3031
2158 of 3031
2159 of 3031
2160 of 3031
2161 of 3031
2162 of 3031
2163 of 3031
2164 of 3031
2165 of 3031
2166 of 3031
2167 of 3031
2168 of 3031
2169 of 3031
2170 of 3031
2171 of 3031
2172 of 3031
2173 of 3031
2174 of 3031
2175 of 3031
2176 of 3031
2177 of 3031
2178 of 3031
2179 of 3031
2180 of 3031
2181 of 3031
2182 of 3031
2183 of 3031
2184 of 3031
2185 of 3031
2186 of 3031
2187 of 3031
2188 of 3031
2189 of 3031
2190 of 3031

2747 of 3031
2748 of 3031
2749 of 3031
2750 of 3031
2751 of 3031
2752 of 3031
2753 of 3031
2754 of 3031
2755 of 3031
2756 of 3031
2757 of 3031
2758 of 3031
2759 of 3031
2760 of 3031
2761 of 3031
2762 of 3031
2763 of 3031
2764 of 3031
2765 of 3031
2766 of 3031
2767 of 3031
2768 of 3031
2769 of 3031
2770 of 3031
2771 of 3031
2772 of 3031
2773 of 3031
2774 of 3031
2775 of 3031
2776 of 3031
2777 of 3031
2778 of 3031
2779 of 3031
2780 of 3031
2781 of 3031
2782 of 3031
2783 of 3031
2784 of 3031
2785 of 3031
2786 of 3031
2787 of 3031
2788 of 3031
2789 of 3031
2790 of 3031
2791 of 3031
2792 of 3031
2793 of 3031
2794 of 3031
2795 of 3031
2796 of 3031
2797 of 3031
2798 of 3031
2799 of 3031
2800 of 3031
2801 of 3031
2802 of 3031
2803 of 3031
2804 of 3031
2805 of 3031
2806 of 3031
2807 of 3031
2808 of 3031
2809 of 3031
2810 of 3031
2811 of 3031
2812 of 3031
2813 of 3031
2814 of 3031
2815 of 3031
2816 of 3031
2817 of 3031
2818 of 3031
2819 of 3031
2820 of 3031
2821 of 3031
2822 of 3031
2823 of 3031

C:\Users\lzell\AppData\Local\Temp\ipykernel_12480\812262311.py:42: RuntimeWarning: Mean of empty slice
  t_avg = np.nanmean(subset_ym['t']).round(6)
C:\Users\lzell\AppData\Local\Temp\ipykernel_12480\812262311.py:43: RuntimeWarning: Mean of empty slice
  p_avg = np.nanmean(subset_ym['p']).round(6)


2950 of 3031
2951 of 3031
2952 of 3031
2953 of 3031
2954 of 3031
2955 of 3031
2956 of 3031
2957 of 3031
2958 of 3031
2959 of 3031
2960 of 3031
2961 of 3031
2962 of 3031
2963 of 3031
2964 of 3031
2965 of 3031
2966 of 3031
2967 of 3031
2968 of 3031
2969 of 3031
2970 of 3031
2971 of 3031
2972 of 3031
2973 of 3031
2974 of 3031
2975 of 3031
2976 of 3031
2977 of 3031
2978 of 3031
2979 of 3031
2980 of 3031
2981 of 3031
2982 of 3031
2983 of 3031
2984 of 3031
2985 of 3031
2986 of 3031
2987 of 3031
2988 of 3031
2989 of 3031
2990 of 3031
2991 of 3031
2992 of 3031
2993 of 3031
2994 of 3031
2995 of 3031
2996 of 3031
2997 of 3031
2998 of 3031
2999 of 3031
3000 of 3031
3001 of 3031
3002 of 3031
3003 of 3031
3004 of 3031
3005 of 3031
3006 of 3031
3007 of 3031
3008 of 3031
3009 of 3031
3010 of 3031
3011 of 3031
3012 of 3031
3013 of 3031
3014 of 3031
3015 of 3031
3016 of 3031
3017 of 3031
3018 of 3031
3019 of 3031
3020 of 3031
3021 of 3031
3022 of 3031
3023 of 3031
3024 of 3031
3025 of 3031
3026 of 3031

In [5]:
# format into df, save
final_df = pd.DataFrame(all_data)

out_path = os.path.join(folder_climate, 'RGI_Monthly_Averages.csv')
final_df.to_csv(out_path, index=False)